In [9]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
from tqdm import trange
import time as tm
import logging
from datetime import datetime

In [10]:
class Crawler:
    def __init__(self, year, month, co_id = " "):
        self.year = str(year)
        self.month = str(month).zfill(2)
        self.co_id = co_id
    
    def start(self, display = False):
        Id, name, date, time, place, message, ch_file, en_file, more_information, video_address, attention = [[] for x in range(11)]
        
        # Create log file
        FORMAT = '%(asctime)s %(levelname)s: %(message)s'
        logging.basicConfig(level = logging.INFO, filename = 'crawler.log', filemode = 'w', format = FORMAT)
        logging.info('Updating data start')
        
        dic_name = datetime.now().strftime("%Y_%m_%d")
        
        # Initialize params
        payload = {
            # 上市 / 未上市
            "encodeURIComponent": "1",
            "step": "1",
            "firstin": "1",
            "off": "1",
            "TYPEK": "sii",
            # 年度
            "year": self.year,
            # 月份
            "month": self.month,
            # 公司代號
            "co_id": self.co_id
        }
        
        # get the response data through post method and params
        response = requests.post("https://mops.twse.com.tw/mops/web/t100sb02_1"
                                 , data = payload)

        soup = BeautifulSoup(response.text, "html.parser")
        
        if(display):
            print("Result: ")
            print(soup.prettify())
        
        result_even = soup.find_all("tr", class_ = "even")
        result_odd = soup.find_all("tr", class_ = "odd")
        result_total = result_even + result_odd
        
        for i in trange(len(result_total)):
            data_td = result_total[i].find_all("td")
            
            for ptr in range(len(data_td)):
                if ptr == 0:
                    Id.append(data_td[ptr].getText())
                elif ptr == 1:
                    name.append(data_td[ptr].getText())
                elif ptr == 2:
                    date.append(data_td[ptr].getText())
                elif ptr == 3:
                    time.append(data_td[ptr].getText())
                elif ptr == 4:
                    place.append(data_td[ptr].getText())
                elif ptr == 5:
                    message.append(data_td[ptr].getText())
                elif ptr == 6:
                    temp = data_td[ptr].getText()
                    
                    if ".pdf" in temp:
                        download_payload = {
                            "step": "9",
                            "filePath": "/home/html/nas/STR/",
                            "fileName": str(temp),
                            "functionName": "t100sb02_1"
                        }
                        
                        while(True):
                            try:
                                download_response = requests.post("https://mops.twse.com.tw/server-java/FileDownLoad"
                                                     , data = download_payload)
                            except:
                                logging.error('Ch file download error')
                        
                            data = download_response.content
                            tm.sleep(1)
                            
                            if(len(data) != 496):
                                break
                        
                        if not os.path.isdir("file"):
                            os.mkdir("file")
                            
                        if not os.path.isdir("./file/" + dic_name):
                                os.mkdir("./file/" + dic_name)
                            
                        if not os.path.isdir("./file/" + dic_name + "/ch"):
                            os.mkdir("./file/" + dic_name + "/ch")
                        
                        with open("./file/" + dic_name + "/ch/" + Id[-1] + "_" + temp, 'wb') as s:
                            s.write(data)
                            
                        ch_file.append(Id[-1] + "_" + temp)
                    else:
                        ch_file.append(temp)
                elif ptr == 7:
                    temp = data_td[ptr].getText()
                    
                    if ".pdf" in temp:
                        download_payload = {
                            "step": "9",
                            "filePath": "/home/html/nas/STR/",
                            "fileName": str(temp),
                            "functionName": "t100sb02_1"
                        }
                        
                        while(True):
                            try:
                                download_response = requests.post("https://mops.twse.com.tw/server-java/FileDownLoad"
                                                     , data = download_payload)
                            except:
                                logging.error('En file download error')
                        
                            data = download_response.content
                            tm.sleep(1)
                            
                            if(len(data) != 496):
                                break
                        
                        if not os.path.isdir("file"):
                            os.mkdir("file")
                            
                        if not os.path.isdir("./file/" + dic_name):
                            os.mkdir("./file/" + dic_name)
                            
                        if not os.path.isdir("./file/" + dic_name + "/en"):
                            os.mkdir("./file/" + dic_name + "/en")
                        
                        with open("./file/" + dic_name + "/en/" + Id[-1] + "_" + temp, 'wb') as s:
                            s.write(data)
                            
                        en_file.append(Id[-1] + "_" + temp)
                    else:
                        en_file.append(temp)
                elif ptr == 8:
                    more_information.append(data_td[ptr].getText())
                elif ptr == 9:
                    temp = data_td[ptr].getText()
                    temp = temp.replace("\r", "")
                    temp = temp.replace("\n", "")
                    temp = temp.replace(".", " ")
                    
                    video_address.append(temp)
                elif ptr == 10:
                    attention.append(data_td[ptr].getText())
        
        df = pd.DataFrame({"Co_ID" : Id, "Co_name" : name, "Date" : date, "Time" : time, "Form" : place, "Message" : message
                          , "CH_file" : ch_file, "EN_file" : en_file, "More information" : more_information, "Video address" : video_address, "Attention" : attention})
        
        logging.info('Updating data completet')
        
        return df

In [11]:
crawler = Crawler(111, 2)
df = crawler.start()

  8%|███▋                                        | 5/60 [00:21<03:52,  4.22s/it]


UnboundLocalError: local variable 'download_response' referenced before assignment